In [7]:
# 패션 MNIST - 훈련세트, 테스트세트
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [8]:
# 모델 생성 함수 - 층 추가 가능하도록
def model_fn(a_layer = None): # 기본값 None
    model = keras.Sequential() # 객체 생성
    model.add(keras.layers.Input(shape=(28,28))) # 입력층
    model.add(keras.layers.Flatten()) # 플랫층 - 2차원을 1차원으로 펼쳐주는 기능
    model.add(keras.layers.Dense(100, activation='relu')) # 은닉층(밀집층) - 특성 100개, 렐루함수 사용
    
    if a_layer:
        model.add(a_layer)
        
    model.add(keras.layers.Dense(10, activation='softmax')) # 출력층 - 소프트맥스 함수 사용 

    return model

In [9]:
model = model_fn(keras.layers.Dropout(0.3))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
train_scaled = train_input / 255
test_scaled = test_input / 255

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대적합 직전 시점에 모델을 파일로 저장 
checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras", save_best_only=True) 

# 과대적합되기 직전까지만 훈련하고 종료 - 조기종료
# patience=2 : 검증(테스트) 세트의 정확도가 2회 이상 올라가지 않으면 조기 종료
# restore_best_weights=True : 가장 최적의 가중치로 복구 
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True) 

# 콜백은 배열형태로 여러개 추가 가능
model.fit(train_scaled, train_target, epochs=20, validation_data=(test_scaled, test_target), callbacks=[checkpoint_cb, early_stopping_cb]) 

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7430 - loss: 0.7337 - val_accuracy: 0.8417 - val_loss: 0.4480
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8435 - loss: 0.4373 - val_accuracy: 0.8480 - val_loss: 0.4169
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8568 - loss: 0.3954 - val_accuracy: 0.8556 - val_loss: 0.3986
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8618 - loss: 0.3743 - val_accuracy: 0.8651 - val_loss: 0.3682
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8686 - loss: 0.3608 - val_accuracy: 0.8606 - val_loss: 0.3800
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8767 - loss: 0.3425 - val_accuracy: 0.8732 - val_loss: 0.3541
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8775 - loss: 0.3409 - val_accuracy: 0.8750 - val_loss: 0.3525
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8795 - loss: 0.3295 - 

In [11]:
model = keras.models.load_model("best-model.keras") # 모델 불러오기

In [12]:
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.8739 - loss: 0.3474


[0.35065457224845886, 0.8737999796867371]

In [13]:
# 조기 종료 시점
early_stopping_cb.stopped_epoch

10